In [1]:
import copy
import numpy as np
from itertools import combinations
from itertools import product
import mdptoolbox

### Training Data 2x3:

In [2]:
import os
filename = "./warehousetraining.txt"
print("Path exists? {}".format(os.path.exists(filename)))

Path exists? True


In [3]:
# read file into a numpy array 
import csv
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter='\t', quoting=csv.QUOTE_NONE)
x = list(reader)
training_data_2_3 = np.array(x).astype("str")
print(training_data_2_3.shape)
print(training_data_2_3)

(12108, 2)
[['store' 'red']
 ['store' 'blue']
 ['store' 'white']
 ...
 ['restore' 'red']
 ['restore' 'red']
 ['restore' 'red']]


### Test Data:

In [4]:
import os
filename = "./warehouseorder.txt"
print("Path exists? {}".format(os.path.exists(filename)))

Path exists? True


In [5]:
import csv
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter='\t', quoting=csv.QUOTE_NONE)
x = list(reader)
test_data_2_3 = np.array(x).astype("str")
print(test_data_2_3.shape)
#print(test_data_2_3)

(60, 2)


### Define States, Actions, Transition probability Matrix and Reward Matrix


In [6]:
import copy
import numpy as np
from itertools import combinations
from itertools import product


class T_R_prepare:
    classes = {'red': 1, 'blue': 2, 'white': 3}
    state_action = {'store': 5, 'restore': 8}
    combinations = None
    actions = np.array([[5, 1], [5, 2], [5, 3], [8, 1], [8, 2], [8, 3]])
    Action_dict = None

    # positions as [#x, #y], initial_training_state as [state_action, classes]
    def __init__(self, initial_training_state, positions, number_of_positions, number_of_classes, training_data,
                 test_data):

        self.num_pos = number_of_positions
        self.pos = positions
        self.num_class = number_of_classes
        self.init_training = initial_training_state
        self.training_data = training_data
        self.test_data = test_data
        self.num_comb = np.power((self.num_class + 1), self.num_pos)
        # choose position were to store or restore item
        self.Action = np.arange(self.num_pos)
        return None

    def Action_dict(self):
        x, y = self.pos
        positions = np.ogrid[1:(x + 1), 1:(y + 1)]
        x, y = positions

        position_elem = []
        for i in range(len(y[0])):
            for j in range(len(x)):
                position_elem.append([x[j][0], y[0][i]])

        position_elem = np.array(position_elem)
        a_dict = dict(zip(self.Action, position_elem))

        return a_dict

    # takes training data in array-format ['store/restore', 'red'] and performs the optimal action on the actual state
    # returning the next state and the performed action - for example: ['store/restore', 'red'] --> [0, 0, 0, 0] --> [1, 0, 0, 0], (1,1)
    # action is choosing position element of: {(1,1), (1,2), (2,1), ...} - position in warehouse
    def store_restore(self, array):

        if (array[len(array) - 2] == 5):
            if (np.isin(0, array[:len(array) - 2])):
                pos = np.sort(np.where(array == 0))[0][0]
                array[pos] = array[len(array) - 1]
            
            else:
                pos = self.num_pos

        else:
            if (np.isin(array[len(array) - 1], array[:len(array) - 2])):
                pos = np.sort(np.where(array == array[len(array) - 1]))[0][0]
                array[pos] = 0
                
            else:
                pos = self.num_pos

        output = array[:len(array) - 2]

        return output, pos

    # iterates through given data and performs store_restore method for every state in the data
    # output: array [[0,0,0,0,5,1],1]       array [[0,0,0,0]
    #                [1,0,0,0,5,2],2]              [1,0,0,0]
    #                [1,2,0,0,5,3],1] ...],        [1,2,0,0] ...]
    def data(self, data):

        states = []
        combinations = []

        states.append(np.concatenate((np.zeros(self.num_pos), self.init_training)).astype(int))
        combinations.append(np.zeros(self.num_pos).astype(int))

        init = np.concatenate((np.zeros(self.num_pos), self.init_training))

        for el in data[-(len(data) - 1):]:
            next_combi, action = self.store_restore(init)
            combinations.append(np.array(next_combi).astype(int))

            next_state_action = [self.state_action[el[0]], self.classes[el[1]]]
            next_state = np.concatenate((next_combi, next_state_action))

            states.append(np.array(action + 1).astype(int))
            states.append(np.array(next_state).astype(int))
            init = next_state

          # appending action which states for the final state
        last_state = copy.copy(states[-1])
        
        _, last_pos = self.store_restore(last_state)
        
        states.append(np.array(last_pos).astype(int))


        states = np.array(states)
        states = states.reshape((int(len(states) / 2), 2))

        combinations = np.array(combinations)
        self.combinations = copy.copy(combinations)

        return states, combinations

    def get_training_data(self):
        return self.data(self.training_data)

    def get_test_data(self):
        return self.data(self.test_data)

    def states(self):
        actions_tile = np.tile(self.actions, (self.num_comb, 1))
        states = product([0, 1, 2, 3], repeat = self.num_pos)
        states = np.array(list(states))
        states = np.repeat(states, 6, axis=0)
        states = np.concatenate((states, actions_tile), axis=1)
        return states

    # computes apriori probabilities of the store/restore actions based on the given training data
    def apriori_probs(self):
        training_states, _ = self.get_training_data()
        apriori = self.actions
        apriori_probs = []
        for el in apriori:
            p = 0

            for i in range(len(training_states)):
                if (el == training_states[i][0][-2:]).all():
                    p = p + 1

            p = p / len(training_states)
            apriori_probs.append(p)

        apriori_probs = np.array(apriori_probs)

        assert np.sum(apriori_probs) == 1

        apriori_probs_dict = dict(zip(self.totuple(apriori), self.totuple(apriori_probs)))

        return apriori_probs_dict

    # computes the conditional probabilities of the store/restore actions based on the given training data and apriori probabilities
    def conditional_probs(self):

        probs = []
        training_prob = np.concatenate((np.repeat(self.actions, 6, axis=0), np.tile(self.actions, (6, 1))),
                                       axis=1).reshape((36, 2, 2))
        training_states, _ = self.get_training_data()

        apriori_dict = self.apriori_probs()
        for el in training_prob:
            p = 0

            for i in range(len(training_states) - 1):
                if ((el[0] == training_states[i][0][-2:]).all() and (el[1] == training_states[i + 1][0][-2:]).all()):
                    p = p + 1

            p = p / np.multiply(len(training_states) - 1, apriori_dict[self.totuple(el[0])])
            probs.append(p)

        probs = np.array(probs)
        probs_check = probs.reshape(6, 6)

        # normieren
        for i in range(6):
            probs_check[i] = probs_check[i] / np.sum(probs_check[i])

        probs = probs_check.reshape(36, 1)
        probs = np.squeeze(probs)
        
        print(np.sum(probs_check, axis=1))
        
        #assert (np.sum(probs_check, axis=1) == 1).all()

        training_prob_dict = dict(zip(self.totuple(training_prob), self.totuple(probs)))

        return training_prob_dict

    # from https://stackoverflow.com/questions/10016352/convert-numpy-array-to-tuple/10016379 - totuple function
    def totuple(self, a):
        try:
            return tuple(self.totuple(i) for i in a)
        except TypeError:
            return a



In [7]:
states_actions_2_3 = T_R_prepare(np.array([5, 1]), [2, 3], 6, 3, training_data_2_3, test_data_2_3)

#### States:

In [8]:
states = states_actions_2_3.states()
print(states)

[[0 0 0 ... 0 5 1]
 [0 0 0 ... 0 5 2]
 [0 0 0 ... 0 5 3]
 ...
 [3 3 3 ... 3 8 1]
 [3 3 3 ... 3 8 2]
 [3 3 3 ... 3 8 3]]


#### Actions:

In [9]:
Action_dict = states_actions_2_3.Action_dict()
Action = states_actions_2_3.Action
print(Action_dict)
print(Action)

{0: array([1, 1]), 1: array([2, 1]), 2: array([1, 2]), 3: array([2, 2]), 4: array([1, 3]), 5: array([2, 3])}
[0 1 2 3 4 5]


#### Training data and probabilities:

In [10]:
training_states, combinations = states_actions_2_3.get_training_data()
print(training_states)

[[array([0, 0, 0, 0, 0, 0, 5, 1]) array(1)]
 [array([1, 0, 0, 0, 0, 0, 5, 2]) array(2)]
 [array([1, 2, 0, 0, 0, 0, 5, 3]) array(3)]
 ...
 [array([0, 0, 1, 1, 1, 0, 8, 1]) array(3)]
 [array([0, 0, 0, 1, 1, 0, 8, 1]) array(4)]
 [array([0, 0, 0, 0, 1, 0, 8, 1]) array(4)]]


In [11]:
apriori_probs = states_actions_2_3.apriori_probs()
print(apriori_probs)

{(5, 1): 0.24686157912124215, (5, 2): 0.12528906508093823, (5, 3): 0.1278493557978196, (8, 1): 0.24686157912124215, (8, 2): 0.12528906508093823, (8, 3): 0.1278493557978196}


In [12]:
conditional_probs = states_actions_2_3.conditional_probs()
print(conditional_probs)

[1. 1. 1. 1. 1. 1.]
{((5, 1), (5, 1)): 0.33021077283372363, ((5, 1), (5, 2)): 0.16995650719304115, ((5, 1), (5, 3)): 0.17698226831716293, ((5, 1), (8, 1)): 0.21278019404483103, ((5, 1), (8, 2)): 0.05185680829708932, ((5, 1), (8, 3)): 0.058213449314151886, ((5, 2), (5, 1)): 0.3355306526038233, ((5, 2), (5, 2)): 0.1852340145023072, ((5, 2), (5, 3)): 0.16216216216216217, ((5, 2), (8, 1)): 0.15029663810151614, ((5, 2), (8, 2)): 0.10481212920237312, ((5, 2), (8, 3)): 0.06196440342781806, ((5, 3), (5, 1)): 0.330749354005168, ((5, 3), (5, 2)): 0.1640826873385013, ((5, 3), (5, 3)): 0.17571059431524547, ((5, 3), (8, 1)): 0.1479328165374677, ((5, 3), (8, 2)): 0.06330749354005168, ((5, 3), (8, 3)): 0.1182170542635659, ((8, 1), (5, 1)): 0.1459170013386881, ((8, 1), (5, 2)): 0.06626506024096386, ((8, 1), (5, 3)): 0.0679384203480589, ((8, 1), (8, 1)): 0.36244979919678716, ((8, 1), (8, 2)): 0.17704149933065597, ((8, 1), (8, 3)): 0.18038821954484605, ((8, 2), (5, 1)): 0.18061964403427813, ((8, 2), (5,

#### Transition Probability Matrix:

In [13]:
num_comb = states_actions_2_3.num_comb
num_pos = states_actions_2_3.num_pos
print(num_comb)
print(num_pos)

4096
6


In [14]:
# computes the transition probability
def P(states_i, states_j, action_k):
    
        
        cond_prob_ij = conditional_probs[((states_i[len(states_i) - 2], states_i[len(states_i) - 1]),
                                           (states_j[len(states_j) - 2], states_j[len(states_j) - 1]))]

        if (states_i[len(states_i) - 2] == 5):
            if (states_i[action_k] == 0                        and
             states_i[len(states_i) - 1] == states_j[action_k] and
            (states_i[:action_k] == states_j[:action_k]).all() and
            (states_i[(action_k + 1):-2] == states_j[(action_k + 1):-2]).all()) or \
            (states_i[action_k] != 0 and (states_i[:-2] == states_j[:-2]).all()):
                P_ij = 1 * cond_prob_ij

            else:
                P_ij = 0

        else:
            if (states_i[action_k] != 0 and states_j[action_k] == 0 and
             states_i[len(states_i) - 1] == states_i[action_k]      and
            (states_i[:action_k] == states_j[:action_k]).all()      and
            (states_i[(action_k + 1):-2] == states_j[(action_k + 1):-2]).all())  or \
            (states_i[action_k] == 0 and (states_i[:-2] == states_j[:-2]).all()) or \
            (states_i[action_k] != 0 and (states_i[:-2] == states_j[:-2]).all()  and
             states_i[len(states_i) - 1] != states_i[action_k]):
                P_ij = 1 * cond_prob_ij

            else:
                P_ij = 0

        return P_ij

In [15]:
T1 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T2 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T3 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T4 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T5 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T6 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)
T7 = np.zeros((num_comb*6, num_comb*6), dtype=np.float16)

# Transition Probability Matrix for action doing nothing
for i in range(num_comb*6):
    for j in range(num_comb*6):
        
        cond_prob_ij = conditional_probs[((states[i][len(states[i]) - 2], states[i][len(states[i]) - 1]),
                                           (states[j][len(states[j]) - 2], states[j][len(states[j]) - 1]))]
        
        if (states[i][:-2] == states[j][:-2]).all():
            T7[i][j] = cond_prob_ij

T = np.vstack(([T1], [T2], [T3], [T4], [T5], [T6], [T7]))
print(T.shape)

KeyboardInterrupt: 

In [ ]:
for k in range(len(Action)):
    for i in range(num_comb*6):
        for j in range(num_comb*6):
            T[k][i][j] = P(states[i], states[j], Action[k])


In [25]:
for i in range(7):
    indx1 = np.where(np.sum(T[i], axis = 1) !=1)
    for j in range(num_comb*6):
        T[i][indx1,j] = T[i][indx1,j]/np.sum(T[i][indx1])

for i in range(7):
    indx1 = np.where(np.sum(T[i], axis = 1) !=1)
    print(indx1)

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


#### Reward:

In [15]:
def r(state_i, action_k):
        
    if action_k < num_pos:
            
        actionk = np.sum(Action_dict[action_k])
            
        if (state_i[-2] == 5):
            if (state_i[action_k] == 0):
                R_i = 1/actionk*10
                
            elif(state_i[:num_pos] != 0).all() or (state_i[action_k] != 0):
                R_i = -100
                
            else:
                R_i = -1

        else:
            if (state_i[action_k] != 0 and state_i[-1] == state_i[action_k]):
                R_i = 1/actionk*10
                                
            elif (state_i[action_k] != 0 and state_i[-1] != state_i[action_k]) or (state_i[:num_pos] == 0).all():
                R_i = -100 
                
            else:
                R_i = -1
        
    else:
            
        if (state_i[-2] == 5):
            if (state_i[:num_pos] == 0).any():
                R_i = -100
                
            elif(state_i[:num_pos] != 0).all():
                R_i = 1
                
            else:
                R_i = -50

        else:
            if (state_i[:num_pos] != 0).any():
                R_i = -50
                                
            elif (state_i[:num_pos] == 0).all():
                R_i = 1
                
            else:
                R_i = -50
            
            
    return R_i

In [16]:
Action_R = np.arange(num_pos+1)
R = np.zeros((num_comb*6, len(Action_R)))

for i in range(num_comb*6):
    for k in range(len(Action_R)):
        R_ik = r(states[i], Action_R[k])
        R[i][k] = R_ik
        
print(R)

[[   5.            3.33333333    3.33333333 ...    2.5
     2.         -100.        ]
 [   5.            3.33333333    3.33333333 ...    2.5
     2.         -100.        ]
 [   5.            3.33333333    3.33333333 ...    2.5
     2.         -100.        ]
 ...
 [-100.         -100.         -100.         ... -100.
  -100.          -50.        ]
 [-100.         -100.         -100.         ... -100.
  -100.          -50.        ]
 [   5.            3.33333333    3.33333333 ...    2.5
     2.          -50.        ]]


### Optimization 2x3

In [17]:
T = np.load('T2x3.npy')

In [18]:
mdptoolbox.util.check(T, R)

In [ ]:
pi = mdptoolbox.mdp.PolicyIteration(T, R, 0.9, max_iter = 10*len(states))
pi.run()
policy_pi = pi.policy


In [ ]:
print(policy_pi)

In [ ]:
vi = mdptoolbox.mdp.ValueIteration(T, R, 0.9, max_iter = 10*len(states))
vi.run()
policy_vi = vi.policy

In [ ]:
print(policy_vi)

### Evaluation 2x3

In [ ]:
states_w_Action = np.concatenate((states, np.expand_dims(policy_vi, axis=1)), axis=1)
print(state_w_Action)

In [ ]:
n=0
test,_ = states_actions_2_3.get_test_data()

for i in range(len(test)):
    for j in range(len(states)):
        
        if ((test[i][0] == states_w_Action[j][:-1]).all() and (test[i][1] == states_w_Action[j][-1]).all()):
            n = n + 1

Accuracy = n/len(test)
print('Accuracy:', Accuracy)

In [ ]:
def search(array, in_array):
    b = 0
    for i in range(len(in_array)):
        if (array == in_array[i][:-1]).all():
            b = [in_array[i][-1]]
            #print(i)
    return b



In [ ]:
import copy

init = copy.copy(states_w_Action[0])
test_states = []
test_states.append(states_w_Action[0])

for i in range(len(test) - 1):

    if (init[-3] == 5):
        if init[-1] < num_pos:
            init[init[-1]] = init[-2]
    else:
        if init[-1] < num_pos:
            init[init[-1]] = 0

    new_state = np.concatenate((init[:num_pos], test[i + 1][0][-2:]))
    new_action = search(new_state, states_w_Action)
    new_state_new_action = np.concatenate((new_state, new_action))
    test_states.append(new_state_new_action)
    init = copy.copy(new_state_new_action)

test_states = np.array(test_states)

In [ ]:
referenz = 0
actual = 0
for i in range(len(test)):
    if test[i][1] > num_pos-1:
        referenz = referenz + 0
    else:
        referenz = referenz + np.sum(Action_dict[int(test[i][1])])

for i in range(len(test_states)):
    if test_states[i][-1] > num_pos-1:
        actual = actual + 0
    else:
        actual = actual + np.sum(Action_dict[test_states[i][-1]])

In [ ]:
print(actual)
print(referenz)
print((referenz-actual)/referenz * 100, '%')

In [ ]:
print(test)

In [ ]:
print(test_states)